In [314]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from itertools import product
from numpy.linalg import cholesky
import plotly.graph_objs as go
import plotly.offline as po
from numpy.linalg import cholesky

from scipy.optimize import basinhopping

In [417]:
assets = pd.read_csv('m_returns.csv', index_col = 'date')
assets.head()

,SPX,EEM,XAU,USTTEN,USDCOP
date,,,,,
2011-02-14,0.046434,-0.057770,-0.038196,-0.021115,0.007020
2011-02-15,0.044507,-0.060353,-0.004829,-0.020181,0.011852
2011-02-16,0.045751,-0.049548,-0.002805,-0.010126,0.019702
2011-02-17,0.050947,-0.035324,0.009087,-0.012194,0.015859
2011-02-18,0.054716,-0.020095,0.014822,-0.018477,0.006955


In [418]:
avgr_5y = assets.rolling(window=365*5).mean().iloc[-1].values
std_5y = assets.rolling(window=365*5).std().iloc[-1].values
cov_5y = assets.rolling(window=365*5).cov().iloc[-5:].values

# MVO

In [425]:
expret = avgr_5y
expvol = std_5y
corr = assets.rolling(window=365*5).corr().iloc[-5:].values

covar = cov_5y
# for i,j in product(expvol,repeat=2):
#     covar.loc[i,j] = corr.loc[i,j]*i*j

    
x0 = [0.25,0.25,0.25,0.25, 0.25]
bnds = ((0, 1), (0, 1),(0, 1),(0, 1), (-0.5, 1.5))

#minimize variance for each level of return
def por_var(x):
    return np.dot(np.matmul(x,covar),x)
def w(x):
    return np.sum(x[:4])-1

eff={}
ws={}
for i in np.arange(min(expret), max(expret)+0.02, 0.0025):
    r = i/2

    def port_ret(x):
        return np.dot(x,expret)-r
    cons= [{'type': 'eq', 'fun': port_ret},{'type': 'eq', 'fun': w}]
    res = minimize(por_var,x0,bounds=bnds,constraints=cons)
    eff[r] = np.sqrt(por_var(res.x))
    ws[r] = res.x

eff = pd.Series(eff)

In [420]:
eff_df = pd.DataFrame({'Std': eff}).mul(100).reset_index()
ws_keys_multiplied = {key * 100: value for key, value in ws.items()}
ws_df = pd.DataFrame(ws_keys_multiplied).mul(100).round(3).T.reset_index()
ws_df.columns = ['Return', 'SPX', 'EEM', 'XAU', 'USTTEN', 'USDCOP']
result_df = pd.concat([eff_df, ws_df], axis=1)
result_df.drop(columns=['index'], inplace=True)
result_df

,Std,Return,SPX,EEM,XAU,USTTEN,USDCOP
0,2.283223,-0.120189,0.026,21.586,1.899,76.488,18.816
1,2.350046,0.004811,0.052,22.450,14.040,63.458,24.728
2,2.386973,0.129811,4.311,20.234,21.212,54.244,27.988
3,2.406289,0.254811,11.085,16.267,25.133,47.516,30.639
4,2.460961,0.379811,17.962,12.696,28.839,40.503,33.069
5,2.547067,0.504811,24.771,8.268,32.751,34.210,36.027
6,2.663166,0.629811,31.587,3.848,36.651,27.914,38.979


In [423]:
#Maximize return for each level of variance 
cons= [{'type': 'eq', 'fun': w}] #constraints for global minimum variance portfolio
gmvp = minimize(por_var,x0,bounds=bnds,constraints=cons) # weights of GMVP
min_v = np.sqrt(por_var(gmvp.x)) # minimum volatility possible
max_v = np.max(expvol)


def mvo(rr,covar):
    eff_v={}
    eff_x={}
    for q in np.arange(min_v-0.003, max_v+0.002, 0.0025):
        def varp(x):
            return np.dot(np.matmul(x,covar),x)-q**2
        def rfun(x):
            return -np.dot(x,rr)
        cons  = [{'type': 'eq', 'fun': varp},{'type': 'eq', 'fun': w}]
        res = minimize(rfun,x0,bounds=bnds,constraints=cons, tol=1e-7)
        print(res.message)

        eff_v[q] = -rfun(res.x)
        eff_x[q] = res.x

    return pd.Series(eff_v),pd.DataFrame(eff_x)

eff_v,xxx= mvo(expret, covar)

Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully


In [426]:
def scat(traces,annots=[]):
  layout = go.Layout(width=550,height=400,margin=dict(l=50,r=20,b=30,t=50),template='plotly_white',
                    annotations=annots,
                    showlegend=False,
                    title={'text':'<b>Frontera Eficiente </b> ','xanchor':'left','font':{'size':10}},
                  )
  fig = go.Figure(data=traces, layout=layout)
  po.init_notebook_mode(connected=True)
  config = {'showLink':False}
  po.iplot(fig,config=config)

     

traces=[]
temp = eff.iloc[2:]
traces.append(go.Scatter(x=temp,y=temp.index,mode='lines+markers',line={'color':'dodgerblue'},marker={'size':6,'color':'blue'}))
traces.append(go.Scatter(x=eff_v.index,y=eff_v,mode='lines+markers',line={'color':'purple'},marker={'size':6,'color':'purple'}))
traces.append(go.Scatter(x=expvol,y=expret,mode='markers',marker={'size':7,'color':'red'},opacity=0.7))
annots = [{'x':temp.loc[i]+0.0021,'y':i,'text':'('+str(temp.round(2).loc[i])+','+str(round(i,2))+')','showarrow':False,'font':{'size':8,'color':'blue'}} for i in temp.index]
scat(traces,annots)

In [308]:
#Solución de minimos locales usando basinhopping
def mvo(rr,covar):
    r_={}
    r_bh={}
    for q in np.arange(min_v-0.003, max_v +0.002, 0.0025):
        def varp(x):
            return np.dot(np.matmul(x,covar),x)-q**2
        def rfun(x):
            return -np.dot(x,rr)
        cons  = [{'type': 'eq', 'fun': varp},{'type': 'eq', 'fun': w}]
        res = minimize(rfun,x0,bounds=bnds,constraints=cons)
        res_ = basinhopping(rfun, x0, minimizer_kwargs={'bounds':bnds,'constraints':cons},niter=200)
        r_[q] = -rfun(res.x)
        r_bh[q] = -rfun(res_.x)

    return pd.Series(r_),pd.Series(r_bh)
r_,r_bh= mvo(expret,covar)

c:\Users\maria\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\optimize\_optimize.py:284: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



In [309]:
def scat(traces,annots=[]):
    layout = go.Layout(width=550,height=400,margin=dict(l=50,r=20,b=30,t=50),template='plotly_white',
                    annotations=annots,
                    showlegend=False,
                    title={'text':'<b>Comparativo minimize vs basinhopping </b> ','xanchor':'left','font':{'size':10}},
                  )
    fig = go.Figure(data=traces, layout=layout)
    po.init_notebook_mode(connected=True)
    config = {'showLink':False}
    po.iplot(fig,config=config)


traces=[]
temp = eff.iloc[2:]
traces.append(go.Scatter(x=r_bh.index,y=r_bh,mode='lines+markers',line={'color':'purple'},marker={'size':6,'color':'purple'}))
traces.append(go.Scatter(x=r_.index,y=r_,mode='lines+markers',line={'color':'blue'},marker={'size':6,'color':'blue'}))
traces.append(go.Scatter(x=expvol,y=expret,mode='markers',marker={'size':7,'color':'red'},opacity=0.7))

scat(traces)

In [310]:
def scat(traces,annots=[]):
    layout = go.Layout(width=550,height=400,margin=dict(l=50,r=20,b=30,t=50),template='plotly_white',
                    annotations=annots,
                    showlegend=False,
                    title={'text':'<b>Frontera Eficiente - Basinhopping </b> ','xanchor':'left','font':{'size':10}},
                  )
    fig = go.Figure(data=traces, layout=layout)
    po.init_notebook_mode(connected=True)
    config = {'showLink':False}
    po.iplot(fig,config=config)
    
traces=[]
temp = eff.iloc[2:]
traces.append(go.Scatter(x=temp,y=temp.index,mode='lines+markers',line={'color':'dodgerblue'},marker={'size':6,'color':'blue'}))
traces.append(go.Scatter(x=r_bh.index,y=r_bh,mode='lines+markers',line={'color':'purple'},marker={'size':6,'color':'purple'}))
traces.append(go.Scatter(x=expvol,y=expret,mode='markers',marker={'size':7,'color':'red'},opacity=0.7))
annots = [{'x':temp.loc[i]+0.0021,'y':i,'text':'('+str(temp.round(2).loc[i])+','+str(round(i,2))+')','showarrow':False,'font':{'size':8,'color':'blue'}} for i in temp.index]

scat(traces,annots)

# REF

In [384]:
#Resample eficient frontier:
# 1. 10k observations of uncorrelated random variables Z~N(0,1)
# 2. Cholesky decompose correlation matrix to generate correlated random variables by transforming Z. 
# 3. correlated returns rij =  Mui + Zij*voli for each asset class i for each iteration j.
# 4. find eficient frontier at each iteration using correlated random returns found in 3. 
# 5. find the mean of the optimized weights at each pair r,vol on the EF. 

n = 200
C = cholesky(corr)
Zu = np.random.normal(size=(len(expret),n))
Zc = np.matmul(C,Zu).transpose()

Mu = np.tile(expret,(n,1))
S = np.tile(expvol,(n,1)) 
R = Mu + -(S**2/2)/100 +np.multiply(Zc,S)

In [385]:
def scat(traces,annots=[]):
    layout = go.Layout(width=550,height=400,margin=dict(l=50,r=20,b=30,t=50),template='plotly_white',
                    annotations=annots,
                    showlegend=True,
                    legend=dict(x=0.7, y=1),
                    title={'text':'<b> Muestra aleatoria y correlacionada </b> ','xanchor':'left','font':{'size':10}},
                  )
    fig = go.Figure(data=traces, layout=layout)
    po.init_notebook_mode(connected=True)
    config = {'showLink':False}
    po.iplot(fig,config=config)
  
traces=[]
traces.append(go.Scatter(x=Zc[:,0],y=Zc[:,3],mode='markers',marker={'color':'red','size':3.5},opacity=0.8, name = 'Aleatorios'))
traces.append(go.Scatter(x=Zu.T[:,0],y=Zu.T[:,3],mode='markers',marker={'color':'black','size':3},opacity=0.8, name = 'Correlacionados'))

print(corr)
scat(traces)

[[ 1.          0.749956    0.13806071 -0.03973987 -0.48620963]
 [ 0.749956    1.          0.30490137  0.02675822 -0.61954999]
 [ 0.13806071  0.30490137  1.          0.46282576 -0.2635806 ]
 [-0.03973987  0.02675822  0.46282576  1.          0.11968904]
 [-0.48620963 -0.61954999 -0.2635806   0.11968904  1.        ]]


In [430]:
ref_=[]
ws_=[]
for i in R:
    ri, wi = mvo(i, covar) 
    ref_.append(ri)
    ws_.append(wi)

Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
Optimization terminated successfully
O

In [431]:
wsdf = pd.concat(ws_,axis=1).transpose()
wsdf.round(3).mean()

0    0.261512
1    0.216616
2    0.256518
3    0.265392
4    0.079306
dtype: float64

In [432]:
ref_df = pd.concat(ref_,axis=1).transpose()

mean_ref = ref_df.mean(axis=0)
traces=[]
temp = eff.iloc[2:]
traces.append(go.Scatter(x=eff_v.index,y=eff_v,mode='lines',line={'color':'purple'}))
traces.append(go.Scatter(x=expvol,y=expret,mode='markers',marker={'size':7,'color':'red'},opacity=0.7))
traces.append(go.Scatter(x=mean_ref.index,y=mean_ref,mode='lines',line={'color':'black'}))
for c in ref_df.columns:
     traces.append(go.Violin(y=ref_df.loc[:,c],jitter=0.2,pointpos=-0.5,
            points='outliers',name=c,side='negative',meanline_visible=True,
            marker={'size':2,'color':'black','opacity':0.4},line={'width':0.5,'color':'gray'}))
scat(traces)

In [434]:
mean_ref

0.021643    0.003218
0.024143    0.022071
0.026643    0.029809
0.029143    0.036052
0.031643    0.041382
0.034143    0.046146
0.036643    0.050250
0.039143    0.054255
0.041643    0.057662
0.044143    0.060373
0.046643    0.062421
0.049143    0.055185
0.051643    0.046452
0.054143    0.040603
0.056643    0.037174
0.059143    0.034309
0.061643    0.031295
0.064143    0.028679
dtype: float64

# Black  Litterman

In [436]:
#BLACK LITTERMAN 
'''
insumos:
* retornos y covar (los mismos del mvo tradicional)
* un vector de recomendaciones q. en esta funcion esta pensado como  un numero de desviaciones. (ej +2 si es un view muy positivo en ese activo)
* un nivel de conviccion en el modelo.. entre mas alto c menos incertidumbre alrededor de los views. (puede estar en funcion del accuarcy del modelo por ejemplo) 
notas adicionales:
tau se puede sensiblizar pero el paper dice que usualmente es algo entre 0.025 y 0.05.. 
pick es una matriz de identidad porque tu siempre tienes views sobre todos los activos. (si tuvieras views sobre solo algunos pick deberia ser una matriz de 0s con 1s unicamente en 
los activos sobre los que se tiene un view...)
resultado:
la funcion devuelve un vector de retornos y covar posterior (osea ya ponderados usando los views y la conviccion que se tiene sobre ellos.)
'''

#BL optim
def BLopt(r,sigma,q,c,tau=0.025):
    pick = pick = np.eye(len(r))
    PEP = np.matmul(np.matmul(pick,covar),pick.T)
    v = r + np.diag(covar**(1/2))*q*np.sqrt(1/12)
    omega = PEP/c
    mu_bl = r + tau*np.matmul(np.matmul(sigma*pick.T,inv(tau*PEP+omega)),v-np.matmul(pick,r))
    sigma_bl = (1+tau)*sigma - tau**2*np.matmul(np.matmul(np.matmul(sigma,pick.T),inv(tau*PEP+omega)),np.matmul(pick,sigma))

    return mu_bl,sigma_bl


In [ ]:
BL